<a href="https://colab.research.google.com/github/holly-hewitt/Abnormal-Infant-Movement-Detection/blob/main/Code/AbnormalPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import tensorflow.compat.v1 as tf
#tf.enable_eager_execution(tf.ConfigProto(log_device_placement=False))
#tf.test.gpu_device_name()

import numpy as np
from sklearn.model_selection import KFold
from itertools import product
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Masking, LSTM, GRU
from sklearn.model_selection import KFold
# import early stopping
from tensorflow.keras.callbacks import EarlyStopping
import pickle
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import class_weight

from tensorflow.keras.layers import Input, Concatenate, Permute, Reshape, Multiply, Lambda, Add
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

from tensorflow.keras.regularizers import l2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def specificity_score(y_true, y_pred):

    # Convert probabilities to binary predictions
    y_pred_bin = np.argmax(y_pred, axis=1)
    y_true_bin = np.argmax(y_true, axis=1)

    tn = np.sum((y_true_bin == 0) & (y_pred_bin == 0))
    fp = np.sum((y_true_bin == 0) & (y_pred_bin != 0))
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    return specificity

def custom_performance_metric(accuracy, specificity, recall, precision, false_positive_rate, accuracy_weight=2):
    """
    Calculate the custom performance metric including precision and with higher weight for accuracy.

    Parameters:
    - accuracy: The accuracy of the model.
    - specificity: The specificity of the model.
    - recall: The recall (sensitivity) of the model.
    - precision: The precision of the model.
    - false_positive_rate: The false positive rate of the model.
    - accuracy_weight: The weight to give to accuracy in the metric calculation.

    Returns:
    - A float representing the custom performance metric.
    """
    # Adjusted calculation to weight accuracy higher
    weighted_accuracy = accuracy * accuracy_weight
    total_weight = accuracy_weight + 1 + 1 + 1  # Adding the implicit weight of 1 for the other metrics
    average_metric = (weighted_accuracy + specificity + recall + precision) / total_weight

    # Subtract the false positive rate
    custom_metric = average_metric - false_positive_rate

    return custom_metric



In [8]:
def train_and_evaluate(dataset_names, create_model_fn, class_weights):

    dataset_results = {}

    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Find best dataset to train and test model on
    #dataset_names = ['X_smoothed_mean_norm']

    for dataset_name in dataset_names:

        # Initialize variables to track the best model
        best_model = None
        best_performance = 0
        best_model_details = ''


         # Load in dataset from pickle
        with open(f'drive/MyDrive/Pickles/{dataset_name}.pickle', 'rb') as handle:
            dataset = pickle.load(handle)

        dataset = np.array(dataset)

        dataset_results[dataset_name] = {}

        print(f'Working on dataset: {dataset_name}')

        accuracies = []
        sensitivities = []
        false_positive_rates = []
        specificities = []
        precisions = []
        performance = []

        fold = 1

        for train_index, test_index in outer_cv.split(dataset):

            # Print current progress
            print(f'Working on fold: {fold}')
            fold += 1

            X_train, X_test = dataset[train_index], dataset[test_index]
            Y_train, Y_test = abnormal_encoded[train_index], abnormal_encoded[test_index]

            X_train = X_train.astype('float32')
            Y_train = Y_train.astype('float32')
            X_test = X_test.astype('float32')
            Y_test = Y_test.astype('float32')

            model = create_model_fn(X_train.shape[1:])
            early_stopping = EarlyStopping(monitor='val_loss', patience=3)

            if class_weights:

                Y_train_classes = np.argmax(Y_train, axis=1)

                # Compute class weights
                cw = class_weight.compute_class_weight('balanced',
                                                    classes=np.unique(Y_train_classes),
                                                    y=Y_train_classes)

                class_weights_dict = dict(enumerate(cw))

                #Fit the model
                print('Fitting model')
                model.fit(X_train, Y_train, epochs=15, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1, class_weight=class_weights_dict)

            else:
                #Fit the model
                print('Fitting model')
                model.fit(X_train, Y_train, epochs=15, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)

            # Predict the test set
            print('Predicting test set')
            Y_pred = model.predict(X_test)

            Y_pred_classes = np.argmax(Y_pred, axis=1)
            Y_test_classes = np.argmax(Y_test, axis=1)

            # Calulate accuracy, sensitivity, false positive rate, specificity and precision
            accuracy = (accuracy_score(Y_test_classes, Y_pred_classes))
            sensitivity = (recall_score(Y_test_classes, Y_pred_classes, average='macro'))
            false_positive_rate = (1 - specificity_score(Y_test, Y_pred))
            specificity = (specificity_score(Y_test, Y_pred))
            precision = (precision_score(Y_test_classes, Y_pred_classes, average='macro'))

            # Calculate the custom performance metric
            model_performance = custom_performance_metric(accuracy, specificity, sensitivity, precision, false_positive_rate)

            # Calulate accuracy, sensitivity, false positive rate, specificity and precision
            accuracies.append(accuracy)
            sensitivities.append(sensitivity)
            false_positive_rates.append(false_positive_rate)
            specificities.append(specificity)
            precisions.append(precision)
            performance.append(model_performance)

            print(f"Model's custom performance metric: {model_performance}")

            # Update the best model if current model is better
            if model_performance > best_performance:
                best_performance = model_performance
                best_model = model
                best_model_details = f'{dataset_name}_fold_{fold}'

        avg_accuracy = np.mean(accuracies)
        avg_sensitivity = np.mean(sensitivities)
        avg_false_positive_rate = np.mean(false_positive_rates)
        avg_specificity = np.mean(specificities)
        avg_precision = np.mean(precisions)
        avg_performance = np.mean(performance)

        std_accuracy = np.std(accuracies)
        std_sensitivity = np.std(sensitivities)
        std_false_positive_rate = np.std(false_positive_rates)
        std_specificity = np.std(specificities)
        std_precision = np.std(precisions)
        std_performance = np.std(performance)

        dataset_results[dataset_name]['Accuracy'] = (avg_accuracy, std_accuracy)
        dataset_results[dataset_name]['Sensitivity'] = (avg_sensitivity, std_sensitivity)
        dataset_results[dataset_name]['False Positive Rate'] = (avg_false_positive_rate, std_false_positive_rate)
        dataset_results[dataset_name]['Specificity'] = (avg_specificity, std_specificity)
        dataset_results[dataset_name]['Precision'] = (avg_precision, std_precision)
        dataset_results[dataset_name]['Performance'] = (avg_performance, std_performance)

        if best_model:
            model_save_path = f'models/best_model_{best_model_details}.keras'
            print(f'Saving best model to {model_save_path}')
            best_model.save(model_save_path)
            print(f'Best Model Details: {best_model_details}, Performance: {best_performance}')

        # Delete dataset to free up memory
        del dataset
        del Y_pred

    for dataset_name, results in dataset_results.items():
        print(f'Dataset: {dataset_name}')
        for metric, (avg, std) in results.items():
            print(f'{metric}: {avg} +/- {std}')
        print('\n')

    return dataset_results

In [20]:
# Model functions

def create_cnn_model(shape, filters=32, kernel_size=3, dropout_rate=0.8):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))  # Adjust the input_shape to match your dataset
    model.add(Conv1D(filters, kernel_size, activation='relu', input_shape=(19301, 16)))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)) )  # Reduced the number of neurons in the dense layer
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_model(shape, lstm_units=32, dropout_rate=0.5, output_classes=3):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))
    model.add(LSTM(lstm_units, return_sequences=False))  # 'return_sequences=False' because we only need the last output
    model.add(Dropout(dropout_rate))
    model.add(Dense(lstm_units, activation='relu'))
    model.add(Dense(output_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_gru_model(shape, gru_units=32, dropout_rate=0.5, output_classes=3):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))
    model.add(GRU(gru_units, return_sequences=False))  # return_sequences=False because we only need the last output
    model.add(Dropout(dropout_rate))
    model.add(Dense(gru_units, activation='relu'))
    model.add(Dense(output_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_cnn_lstm_model(shape, filters=32, kernel_size=3, lstm_units=64, dropout_rate=0.5, output_classes=3):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(LSTM(lstm_units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(lstm_units, activation='relu'))
    model.add(Dense(output_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_cnn_gru_model(shape, filters=32, kernel_size=3, gru_units=64, dropout_rate=0.5, output_classes=3):
    model = Sequential()
    model.add(Masking(mask_value=0., input_shape=shape))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(GRU(gru_units, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(gru_units, activation='relu'))
    model.add(Dense(output_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_lstm_attention_model(shape, lstm_units=64, dropout_rate=0.5, output_classes=3):
    inputs = Input(shape=shape)
    lstm_out = LSTM(lstm_units, return_sequences=True)(inputs)
    attention = Dense(1, activation='tanh')(lstm_out)
    attention = Flatten()(attention)
    attention = Activation('softmax')(attention)
    attention = RepeatVector(lstm_units)(attention)
    attention = Permute([2, 1])(attention)
    sent_representation = Multiply()([lstm_out, attention])
    sent_representation = Lambda(lambda xin: K.sum(xin, axis=-2), output_shape=(lstm_units,))(sent_representation)
    dropout = Dropout(dropout_rate)(sent_representation)
    dense = Dense(lstm_units, activation='relu')(dropout)
    outputs = Dense(output_classes, activation='softmax')(dense)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model



In [5]:
 # Load in dataset from pickle
with open('drive/MyDrive/Pickles/abnormal_encoded.pickle', 'rb') as handle:
    abnormal_encoded = pickle.load(handle)
dataset_results = {'X_smoothed_mean_norm_month': {}, 'X_smoothed_median_norm_month': {}, 'X_smoothed_mean_norm': {}, 'X_smoothed_median_norm': {}}

In [16]:
cnn_dataset_result_cw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_cnn_model, True)


Working on dataset: X_smoothed_mean_norm
Working on fold: 1
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 73ms/step - loss: 64.3880 - accuracy: 0.5686 - val_loss: 51.4076 - val_accuracy: 0.2692
Epoch 2/15
7/7 [==============================] - 0s 31ms/step - loss: 17.2277 - accuracy: 0.6471 - val_loss: 35.9825 - val_accuracy: 0.7308
Epoch 3/15
7/7 [==============================] - 0s 31ms/step - loss: 11.8004 - accuracy: 0.6961 - val_loss: 19.0327 - val_accuracy: 0.6154
Epoch 4/15
7/7 [==============================] - 0s 33ms/step - loss: 6.8147 - accuracy: 0.8529 - val_loss: 29.8845 - val_accuracy: 0.7308
Epoch 5/15
7/7 [==============================] - 0s 30ms/step - loss: 2.4500 - accuracy: 0.9804 - val_loss: 19.6272 - val_accuracy: 0.7308
Epoch 6/15
7/7 [==============================] - 0s 35ms/step - loss: 2.1828 - accuracy: 0.9608 - val_loss: 23.0009 - val_accuracy: 0.6923
Predicting test set
2/2 [==============================] - 0s 8ms/step
Model's cust

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 2s 185ms/step - loss: 47.2806 - accuracy: 0.4854 - val_loss: 9.2705 - val_accuracy: 0.6538
Epoch 2/15
7/7 [==============================] - 0s 32ms/step - loss: 10.0177 - accuracy: 0.6602 - val_loss: 10.9711 - val_accuracy: 0.4615
Epoch 3/15
7/7 [==============================] - 0s 32ms/step - loss: 5.4920 - accuracy: 0.7670 - val_loss: 12.7735 - val_accuracy: 0.7692
Epoch 4/15
7/7 [==============================] - 0s 32ms/step - loss: 5.4368 - accuracy: 0.8738 - val_loss: 17.4143 - val_accuracy: 0.5385
Predicting test set
1/1 [==============================] - 0s 109ms/step
Model's custom performance metric: -0.06999999999999995
Working on fold: 4
Fitting model
Epoch 1/15
7/7 [==============================] - 2s 94ms/step - loss: 56.4266 - accuracy: 0.5146 - val_loss: 117.9981 - val_accuracy: 0.3077
Epoch 2/15
7/7 [==============================] - 0s 40ms/step - loss: 51.5782 - accuracy: 0.5728 - val_loss: 86.2441 - 

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 2s 67ms/step - loss: 44.7753 - accuracy: 0.4272 - val_loss: 2.3558 - val_accuracy: 0.3077
Epoch 2/15
7/7 [==============================] - 0s 34ms/step - loss: 2.6432 - accuracy: 0.4175 - val_loss: 3.0114 - val_accuracy: 0.3077
Epoch 3/15
7/7 [==============================] - 0s 33ms/step - loss: 3.1443 - accuracy: 0.4175 - val_loss: 3.3281 - val_accuracy: 0.3077
Epoch 4/15
7/7 [==============================] - 0s 37ms/step - loss: 3.3545 - accuracy: 0.4175 - val_loss: 3.4045 - val_accuracy: 0.3077
Predicting test set
1/1 [==============================] - 0s 89ms/step
Model's custom performance metric: -0.759375
Working on fold: 5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
7/7 [==============================] - 1s 67ms/step - loss: 54.6418 - accuracy: 0.5146 - val_loss: 37.5831 - val_accuracy: 0.6923
Epoch 2/15
7/7 [==============================] - 0s 38ms/step - loss: 16.0118 - accuracy: 0.6796 - val_loss: 46.2857 - val_accuracy: 0.7692
Epoch 3/15
7/7 [==============================] - 0s 37ms/step - loss: 14.2701 - accuracy: 0.7573 - val_loss: 19.3205 - val_accuracy: 0.6154
Epoch 4/15
7/7 [==============================] - 0s 37ms/step - loss: 4.1637 - accuracy: 0.9126 - val_loss: 33.5229 - val_accuracy: 0.7692
Epoch 5/15
7/7 [==============================] - 0s 32ms/step - loss: 2.0522 - accuracy: 0.9903 - val_loss: 23.9545 - val_accuracy: 0.7308
Epoch 6/15
7/7 [==============================] - 0s 31ms/step - loss: 2.2402 - accuracy: 0.9612 - val_loss: 27.7600 - val_accuracy: 0.7308
Predicting test set
1/1 [==============================] - 0s 90ms/step
Model's custom performance metric: 0.4343807167491378
Saving best model

In [ ]:
cnn_dataset_result_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_cnn_model, False)


In [ ]:
lstm_dataset_results_cw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_lstm_model, True)


In [ ]:
lstm_dataset_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_lstm_model, False)


In [ ]:
gru_dataset_results_cw = c(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_gru_model, True)


In [ ]:
gru_dataset_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'], create_gru_model, False)


In [ ]:
cnn_lstm_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'],create_cnn_lstm_model,False)

In [ ]:
cnn_gru_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'],create_cnn_gru_model,False)

In [ ]:
lstm_attention_results_ncw = train_and_evaluate(['X_smoothed_mean_norm', 'X_smoothed_mean_norm_month','X_smoothed_median_norm', 'X_smoothed_median_norm_month'],create_lstm_attention_model,False)

In [18]:
from sklearn.model_selection import train_test_split

def train_and_evaluate_warped(dataset_names, create_model_fn, class_weights):

    dataset_results = {}

    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

    # Find best dataset to train and test model on
    #dataset_names = ['X_smoothed_mean_norm']

    for dataset_name in dataset_names:

        # Initialize variables to track the best model
        best_model = None
        best_performance = 0
        best_model_details = ''


         # Load in warped dataset from pickle
        with open(f'drive/MyDrive/Pickles/{dataset_name}.pickle', 'rb') as handle:
            dataset = pickle.load(handle)

        # Load in smoothed mean dataset from pickle
        with open(f'drive/MyDrive/Pickles/X_smoothed_mean_norm.pickle', 'rb') as handle:
            unwarped = pickle.load(handle)

        dataset = np.array(dataset)
        unwarped = np.array(unwarped)

        dataset_results[dataset_name] = {}

        print(f'Working on dataset: {dataset_name}')

        accuracies = []
        sensitivities = []
        false_positive_rates = []
        specificities = []
        precisions = []
        performance = []

        fold = 0

        unwarped_split = list(outer_cv.split(unwarped))

        for train_index, test_index in outer_cv.split(dataset):

            unwarped_train_index, unwarped_test_index = unwarped_split[fold]

            # Print current progress
            print(f'Working on fold: {fold}')
            fold += 1

            X_train, X_test_warped = dataset[train_index], dataset[test_index]
            Y_train, Y_test_warped = abnormal_encoded_warped[train_index], abnormal_encoded_warped[test_index]

            X_test = unwarped[unwarped_test_index]
            Y_test = abnormal_encoded[unwarped_test_index]

            X_train_unwarped, X_val, Y_train_unwarped, Y_val = train_test_split(
                unwarped[unwarped_train_index], abnormal_encoded[unwarped_train_index], test_size=0.2, random_state=42)

            X_train = X_train.astype('float32')
            Y_train = Y_train.astype('float32')
            X_test = X_test.astype('float32')
            Y_test = Y_test.astype('float32')

            model = create_model_fn(X_train.shape[1:])
            early_stopping = EarlyStopping(monitor='val_loss', patience=3)

            if class_weights:

                Y_train_classes = np.argmax(Y_train, axis=1)

                # Compute class weights
                cw = class_weight.compute_class_weight('balanced',
                                                    classes=np.unique(Y_train_classes),
                                                    y=Y_train_classes)

                class_weights_dict = dict(enumerate(cw))

                #Fit the model
                print('Fitting model')
                model.fit(X_train, Y_train, epochs=15, batch_size=16, callbacks=[early_stopping], verbose=1, class_weight=class_weights_dict, validation_data = (X_val, Y_val))

            else:
                #Fit the model
                print('Fitting model')
                model.fit(X_train, Y_train, epochs=15, batch_size=16,  callbacks=[early_stopping], verbose=1, validation_data = (X_val, Y_val))

            # Predict the test set
            print('Predicting test set')
            Y_pred = model.predict(X_test)

            Y_pred_classes = np.argmax(Y_pred, axis=1)
            Y_test_classes = np.argmax(Y_test, axis=1)

            # Calulate accuracy, sensitivity, false positive rate, specificity and precision
            accuracy = (accuracy_score(Y_test_classes, Y_pred_classes))
            sensitivity = (recall_score(Y_test_classes, Y_pred_classes, average='macro'))
            false_positive_rate = (1 - specificity_score(Y_test, Y_pred))
            specificity = (specificity_score(Y_test, Y_pred))
            precision = (precision_score(Y_test_classes, Y_pred_classes, average='macro'))

            # Calculate the custom performance metric
            model_performance = custom_performance_metric(accuracy, specificity, sensitivity, precision, false_positive_rate)

            # Calulate accuracy, sensitivity, false positive rate, specificity and precision
            accuracies.append(accuracy)
            sensitivities.append(sensitivity)
            false_positive_rates.append(false_positive_rate)
            specificities.append(specificity)
            precisions.append(precision)
            performance.append(model_performance)

            print(f"Model's custom performance metric: {model_performance}")

            # Update the best model if current model is better
            '''if model_performance > best_performance:
                best_performance = model_performance
                best_model = model
                best_model_details = f'{dataset_name}_fold_{fold}'''

        avg_accuracy = np.mean(accuracies)
        avg_sensitivity = np.mean(sensitivities)
        avg_false_positive_rate = np.mean(false_positive_rates)
        avg_specificity = np.mean(specificities)
        avg_precision = np.mean(precisions)
        avg_performance = np.mean(performance)

        std_accuracy = np.std(accuracies)
        std_sensitivity = np.std(sensitivities)
        std_false_positive_rate = np.std(false_positive_rates)
        std_specificity = np.std(specificities)
        std_precision = np.std(precisions)
        std_performance = np.std(performance)

        dataset_results[dataset_name]['Accuracy'] = (avg_accuracy, std_accuracy)
        dataset_results[dataset_name]['Sensitivity'] = (avg_sensitivity, std_sensitivity)
        dataset_results[dataset_name]['False Positive Rate'] = (avg_false_positive_rate, std_false_positive_rate)
        dataset_results[dataset_name]['Specificity'] = (avg_specificity, std_specificity)
        dataset_results[dataset_name]['Precision'] = (avg_precision, std_precision)
        dataset_results[dataset_name]['Performance'] = (avg_performance, std_performance)

        '''if best_model:
            model_save_path = f'models/best_model_{best_model_details}.keras'
            print(f'Saving best model to {model_save_path}')
            best_model.save(model_save_path)
            print(f'Best Model Details: {best_model_details}, Performance: {best_performance}')'''

        # Delete dataset to free up memory
        del dataset
        del Y_pred

    for dataset_name, results in dataset_results.items():
        print(f'Dataset: {dataset_name}')
        for metric, (avg, std) in results.items():
            print(f'{metric}: {avg} +/- {std}')
        print('\n')

    return dataset_results

In [ ]:
 # Load in dataset from pickle
with open('drive/MyDrive/Pickles/labels_magnitude_warped.pickle', 'rb') as handle:
    abnormal_encoded_warped = pickle.load(handle)



# Use the sklearn OneHotEncoder to one-hot encode the data
from sklearn.preprocessing import OneHotEncoder

# Create the OneHotEncoder object
onehot_encoder = OneHotEncoder(sparse=False)

# One-hot encode the data
abnormal_encoded_warped = onehot_encoder.fit_transform(np.array(abnormal_encoded_warped).reshape(-1, 1))

print(abnormal_encoded_warped)

In [21]:
warped_cnn_dataset_result_ncw = train_and_evaluate_warped(['X_magnitude_warped'], create_cnn_model, False)


Working on dataset: X_magnitude_warped
Working on fold: 0
Fitting model
Epoch 1/15
17/17 [==============================] - 2s 81ms/step - loss: 12.9067 - accuracy: 0.6187 - val_loss: 160.0587 - val_accuracy: 0.6538
Epoch 2/15
17/17 [==============================] - 0s 22ms/step - loss: 3.3571 - accuracy: 0.8988 - val_loss: 158.9882 - val_accuracy: 0.5769
Epoch 3/15
17/17 [==============================] - 0s 22ms/step - loss: 2.0006 - accuracy: 0.9689 - val_loss: 150.6693 - val_accuracy: 0.6154
Epoch 4/15
17/17 [==============================] - 0s 22ms/step - loss: 1.6626 - accuracy: 0.9844 - val_loss: 144.0360 - val_accuracy: 0.6923
Epoch 5/15
17/17 [==============================] - 0s 21ms/step - loss: 1.4010 - accuracy: 0.9883 - val_loss: 135.3915 - val_accuracy: 0.6923
Epoch 6/15
17/17 [==============================] - 0s 22ms/step - loss: 1.1464 - accuracy: 1.0000 - val_loss: 128.4753 - val_accuracy: 0.6923
Epoch 7/15
17/17 [==============================] - 0s 22ms/step - lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fitting model
Epoch 1/15
17/17 [==============================] - 2s 38ms/step - loss: 9.6305 - accuracy: 0.5992 - val_loss: 115.4482 - val_accuracy: 0.6538
Epoch 2/15
17/17 [==============================] - 0s 22ms/step - loss: 2.3849 - accuracy: 0.9144 - val_loss: 102.8027 - val_accuracy: 0.2692
Epoch 3/15
10/17 [================>.............] - ETA: 0s - loss: 2.0414 - accuracy: 0.9625

KeyboardInterrupt: 